<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>

# Laboratory work #3. FHIR resources Practitioner and Schedule

A clinical coordinator invited John for a visit. John said, he’d like to visit the center on Monday if possible. The coordinator needs to select an appropriate doctor with available slots in the schedule. So, the coordinator views a list of available schedules with a `General Practice` service type.

Select the first found schedule with available slots for the next Monday (09-16-2018).

## 1 Introduction

### 1.1 Import library

At the beginning we need to import libraries `fhirpy` and `os`.  
Also, import `pprint` helper function from local `utils.py` file. We'll use `pprint` to display some structures.

In [ ]:
import os
import fhirpy

from utils import pprint

### 1.2 Create an instance of connection

To load data from FHIR server we should instantiate `FHIRClient` class from `fhirpy` package.  
Let's pass `url` and `authorization` arguments from environment.

In [ ]:
client = fhirpy.FHIRClient(
    fhir_version='4.0.0',
    url=os.environ['BOX_URL'] + '/fhir', 
    authorization=os.environ['BOX_AUTHORIZATION'])

Now, we are able to operate with FHIR resources using `client`.

## 2 FHIR resources Practitioner, Schedule and Slot

### 2.1 [Practitioner](https://www.hl7.org/fhir/practitioner.html)

`Practitioner` covers all individuals who are engaged in the healthcare process and healthcare-related services as part of their formal responsibilities.

The two major distinctions from [RelatedPerson](https://www.hl7.org/fhir/relatedperson.html) are:
* formal responsibility for the healthcare-related services
* the fact that `Practitioner` operates on behalf of the care delivery orgranization over multiple patients.

Within an organization `Practitioner` can be associated with multiple roles defined by [PractitionerRole](https://www.hl7.org/fhir/practitionerrole.html).

Clinical specialty of the clinician is defined in terms of [Practice Setting Code Value Set](https://www.hl7.org/fhir/valueset-c80-practice-codes.html).

### 2.2 [Schedule](https://www.hl7.org/fhir/schedule.html)

`Schedule` controls dates and times available for performance of a service and/or the use of a resource.

The service provided by `Schedule` can be parameterized with `serviceCategory`, `serviceType` and `speciality`.

* [serviceCategory](https://www.hl7.org/fhir/valueset-service-category.html)
  Is a broader categorisation of the service that will be performed during this appointment.  
  For example, in [service category system](http://hl7.org/fhir/service-category) code `2` stands for `Aged Care`. 

* [serviceType](https://www.hl7.org/fhir/valueset-service-type.html)
  Is used for more concrete service that will be performed during this appointment.  
  For example, in [service type system](http://hl7.org/fhir/service-type) `Aged Care` is split into multiple
  components:  
  * code `2` - `Aged Care Assessment`
  * code `3` - `Aged Care Information/Referral`
  * code `4` - `Aged Residential Care`.

* [speciality](https://www.hl7.org/fhir/valueset-c80-practice-codes.html)
  The speciality of a practitioner that would be required to perform the service requested.  
  For example, in [practice codes](https://www.hl7.org/fhir/valueset-c80-practice-codes.html)
  for snomed system `http://snomed.info/sct` code `394814009` stands for `General practice`.

The only required field is `actor` - the resource the `Schedule` is providing availability information for.  
It is of `Reference` type and can reffer to `Practitioner`, `PractitionerRole`, `Device` or a few other resources.

There are five [search parameters](https://www.hl7.org/fhir/schedule.html#search) defined for `Schedule` resource
* `active` to find a `Schedule` in a concrete state (active or inactive)
* `actor` to find a `Schedule` for the individual (`Practitioner`, for example).
* `identifier` to search for a `Schedule` with specific `identifier`
* `type` for the type of appointments that can be booked into associated slots.
* `date` for `Schedule` resources that have a period that contains this date specified.

A slot of time on a schedule that might be available for booking appointments is defined with [Slot](https://www.hl7.org/fhir/slot.html) resource.

`Schedule` does not directly contains `Slot` resources.
It has `planningHorizon` instead - the interval for which an organization is accepting appointments and each `Slot` has a reference to the `Schedule` to which it belongs.

Let's suppose that the organization has a weekly schedule with eight one-hour appointments permitted per a business day.  
The weekly schedule will fit in a single `Schedule` resource with the dates for the start and end of week set.  
This `Schedule` resource will then have 40 `Slot` resources associated with it.

### 2.3 [Slot](https://www.hl7.org/fhir/slot.html)

`Slot`s are effectively spaces of free/busy time, they do not provide any information about appointments that are available, just the time, and optionally what the time can be used for.

There can be multiple `Slot`s reffering to a given instant of time, this is possible in a case of, for example, open group therapy.

The slots in a schedule are not necessarily of the same size, and can be different for different days of week.

## 3 Find a schedule

Let's search for a schedule with `General Practice` type of appointments. 

`Schedule` has a [search parameter](https://www.hl7.org/fhir/schedule.html#search) `service-type` that translates into `serviceType` field. `serviceType` is defined in terms of [ServiceType value set](https://www.hl7.org/fhir/valueset-service-type.html).  
On the value set `General Practice` is defined with code `124`.

Note that every `Schedule` provides availability information for a list of resources. The list is stored in a field `actor`.  
We may want to ask `FHIR` server to include those actors in the query result.

In [ ]:
schedules = client.resources('Schedule').search(date='2018-09-16', **{'service-type': 124}).include('Schedule', 'actor').fetch()
schedules

Let's look into the `actors` returned with the query result

In [ ]:
[[actor.to_resource() for actor in schedule['actor']] for schedule in schedules]

Each of the schedules provides availability information for `Practitioner` and `Location`.

## 4 Load slots schedules

Please, read about [Slot search parameters](https://www.hl7.org/fhir/slot.html#search) before moving on.

Now let's search for slots defined for the schedules.

In [ ]:
[client.resources('Slot').search(schedule=schedule['id']).fetch() for schedule in schedules]

There are no available slots for `Adam Ainslay` on Monday 09-16-2018, so we'll work with `Kelly Smith` (first in the schedules list).

In [ ]:
schedule = schedules[0]
schedule

Let's display the schedule in more comfortable format.

In [ ]:
slots = client.resources('Slot').search(schedule=schedule['id']).fetch()
[(slot.get('start'), slot.get('end'), slot.get('status'))  for slot in slots]

**Exercise**: Find all of the `free` slots for the schedule using API. Hint: this can be done with `status` [search parameter](https://www.hl7.org/fhir/slot.html#search).

In [ ]:
slots = client.resources('Slot').search(
    schedule=schedule['id'],
    # Write code here
).fetch()
[(slot.get('start'), slot.get('end'), slot.get('status')) for slot in slots]

**Exercise**: Find all of the slots starting at 11am on Monday 09-16-2018. Hint: use `start` [search parameter](https://www.hl7.org/fhir/slot.html#search).

In [ ]:
slots = client.resources('Slot').search(
    schedule=schedule['id'],
    # Write code here
).fetch()
[(slot.get('start'), slot.get('end'), slot.get('status'))  for slot in slots]

## Summary

In this laboratory work the following topics were covered:
* `Practitioner`, `Schedule` and `Slot` resources
* How to find a `Schedule` by service type and date
* How to load `Slots` for a `Schedule` resource
* How to search `Slots` for specific date and speciality

<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>